In [1]:
#!/usr/bin/env python3
import gymnasium as gym

from collections import namedtuple
import numpy as np

from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.9

In [4]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space,
                          gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape, dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [5]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [6]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs, _ = env.reset()
    sm = nn.Softmax(dim=1).to(device)
    while True:
        obs_v = torch.FloatTensor([obs]).to(device)
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.detach().cpu().numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, truncated, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done or truncated:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs, _ = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [7]:
def filter_batch(batch, percentile):
    filter_fun = lambda s: s.reward * (GAMMA ** len(s.steps))
    disc_rewards = list(map(filter_fun, batch))
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs = []
    train_act = []
    elite_batch = []
    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation,
                                 example.steps))
            train_act.extend(map(lambda step: step.action,
                                 example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound

In [8]:
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1", desc=generate_random_map(size=4), is_slippery=False))

# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

D = obs_size
H = 16
O = n_actions

# net = create_deterministic_model(obs_size, 16, n_actions).to(device)



In [9]:
%load_ext autoreload
%autoreload 2
from lib.layers import Residual, Unpack
from lib.quantumsearch import FitnessFunction, OneToManyNetwork, QuantumSearch
from lib.quantumsearch import TransitionFunction


encoder = nn.Sequential(
    nn.Linear(D, H),
    nn.ReLU(),
)

search = QuantumSearch(
    transition=TransitionFunction(
        OneToManyNetwork(
            nn.Sequential(
                nn.Linear(H, H),
                nn.ReLU(),
                nn.Linear(H, 3*H),
                Unpack(3) # Batch, ...,  3 * H -> Batch, ..., H, 3
            )
        ),
    ),
    fitness=FitnessFunction(
        OneToManyNetwork(
            nn.Sequential(
                nn.Linear(H, H),
                nn.ReLU(),
                nn.Linear(H, 1),
                Unpack(1) # Batch, ...,  3 * H -> Batch, ..., 1, 3
            )
        ),
    ),
    max_depth=3,
    beam_width = 1,
    branching_width=3
)

decoder = nn.Sequential(
    nn.Linear(H, O)
)

model = nn.Sequential(
    encoder,
    Residual(search),
    decoder
)

model.to(device)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.01)
writer = SummaryWriter(comment="-frozenlake-search")


In [10]:
'number of parameters', sum(p.numel() for p in model.parameters())

('number of parameters', 1717)

In [11]:
full_batch = []

gamma = 0.995

with torch.device(device):
    for iter_no, batch in enumerate(iterate_batches(env, model, BATCH_SIZE)):

        reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))

        full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)

        if not full_batch:
            continue

        obs_v = torch.FloatTensor(obs).to(device)
        acts_v = torch.LongTensor(acts).to(device)
        full_batch = full_batch[-500:]

        optimizer.zero_grad()

        action_scores_v = model(obs_v)

        loss_v = objective(action_scores_v, acts_v)

        loss_v.backward()
        optimizer.step()

        print("%d: loss=%.3f, rw_mean=%.3f, rw_bound=%.6f, batch=%d" % (
            iter_no, loss_v.item(), reward_mean, reward_bound, len(full_batch))
        )

        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_mean", reward_mean, iter_no)
        writer.add_scalar("reward_bound", reward_bound, iter_no)

        if reward_mean > 0.8:
            print("Solved!")
            break

    writer.close()

/tmp/ipykernel_82462/3986369348.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  obs_v = torch.FloatTensor([obs]).to(device)


0: loss=1.379, rw_mean=0.110, rw_bound=0.000000, batch=11
1: loss=1.327, rw_mean=0.200, rw_bound=0.000000, batch=31
2: loss=1.299, rw_mean=0.460, rw_bound=0.000000, batch=77
3: loss=1.239, rw_mean=0.600, rw_bound=0.030285, batch=124
4: loss=1.197, rw_mean=0.530, rw_bound=0.064611, batch=155
5: loss=1.149, rw_mean=0.580, rw_bound=0.109419, batch=177
6: loss=1.094, rw_mean=0.620, rw_bound=0.185302, batch=189
7: loss=1.036, rw_mean=0.550, rw_bound=0.254187, batch=184
8: loss=0.964, rw_mean=0.650, rw_bound=0.313811, batch=178
9: loss=0.888, rw_mean=0.610, rw_bound=0.348678, batch=186
10: loss=0.812, rw_mean=0.510, rw_bound=0.387420, batch=191
11: loss=0.688, rw_mean=0.600, rw_bound=0.430467, batch=162
12: loss=0.590, rw_mean=0.730, rw_bound=0.478297, batch=133
14: loss=0.646, rw_mean=0.740, rw_bound=0.387420, batch=63
15: loss=0.566, rw_mean=0.700, rw_bound=0.430467, batch=112
16: loss=0.459, rw_mean=0.860, rw_bound=0.478297, batch=130
Solved!
